In [3]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import pandas as pd

In [4]:
username = "capstone_admin"
password = "Energy#2023"
host = "capstonedb.postgres.database.azure.com"
port = "5432"
database = "postgres"

In [6]:
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}")

Base = declarative_base()

class Consumer(Base):
    __tablename__ = 'Consumer'
    Consumer_Number = Column(Integer, primary_key=True)

Base.metadata.create_all(engine)

/var/folders/k8/8klrh8hn75v3fz7g2tjzs0r80000gn/T/ipykernel_20587/1358546259.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
df_Consumer = pd.DataFrame({
    'Consumer_Number': list(range(1, 2809))
})

df_Consumer.to_sql('Consumer', engine, if_exists='replace', index=False)

808

In [ ]:
# Define the second table (df2) with a foreign key referencing the first table's primary key
class Consumption_History(Base):
    __tablename__ = 'table2'
    id = Column(Integer, primary_key=True)
    column3 = Column(String)
    column4 = Column(Integer)
    table1_id = Column(Integer, ForeignKey('table1.id'))
    
    # Define a relationship to access Table1 data
    table1 = relationship('Table1', back_populates='table2')

# Define the relationship from Table1 to Table2
Table1.table2 = relationship('Table2', back_populates='table1')